# **BASELINE - UNET**

# Architecture

Type info here

# Import Dependencies

In [97]:
# File Support
try: 
    import pydicom as dcm
except:
    # Use try except for Google Colab
    !pip install pydicom
from pydicom.data import get_testdata_files
import xml
import xml.etree.ElementTree as ET 

# Base
import numpy as np
import pandas as pd
import json
import random
import gc
from tqdm import tqdm

# Visualization
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.path import Path
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import seaborn as sns
from PIL import Image

# SK-learn
import sklearn
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV

# Files
import os
from os.path import join, split
from glob import glob

# Torch
import torch
from torch import nn
from torch.nn import Conv2d, MaxPool2d, ReLU, ConvTranspose2d
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import torchvision
from torchvision.transforms import CenterCrop
import torch.nn.functional as F
from torch.optim import AdamW

import tempfile
import torch.multiprocessing as mp
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed.fsdp import CPUOffload, wrap

In [98]:
torch.__version__

'2.0.0'

# Config

In [99]:
len(os.listdir("Numpy Dataset\\all\images"))

3062

In [100]:
class CFG:
    random_seed = 42
    gated = True
    path = "Coronary CT Data\Gated_release_final" if gated else "Coronary CT Data/deidentified_nongated"
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    train_new = True
    model_num = len(os.listdir("Models")) if train_new else len(os.listdir("Models"))-1

    batch_size = 8
    nEpochs = 10
    lr = 0.1

    TH = 0.8

    model_name = f"baseline_{model_num}"

In [101]:
import warnings
warnings.filterwarnings("ignore")

In [102]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

In [103]:
torch.cuda.empty_cache()
gc.collect()

2518

In [104]:
def setup(rank, world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'

    # initialize the process group
    dist.init_process_group("gloo", rank=rank, world_size=world_size)

def cleanup():
    dist.destroy_process_group()

## Reproducibility

In [105]:
def set_seed(seed=CFG.random_seed):
    print(f"Seed: {seed}")
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

set_seed()

Seed: 42


# Load Data

In [106]:
def parseXML(xmlfile): 
    # create element tree object 
    tree = ET.parse(xmlfile) 

    all_images = []

    images = tree.find("dict").find("array")
    images = images.findall("dict")

    # Images
    for image in images:
        image_data = {}
        arr = [i.text for i in image if i.tag not in ["array", "dict"]]
        
        for i in range(len(arr)//2):
            image_data[arr[2*i]] = arr[2*i+1]

        image_data['ROIs'] = []

        # ROI
        all_roi = image.find("array").findall('dict')
        for roi in all_roi:
            roi_data = {}
            arr = [i.text for i in roi if i.tag not in ["array", "dict"]]
        
            for i in range(len(arr)//2):
                roi_data[arr[2*i]] = arr[2*i+1]

            all_points = roi.findall('array')
            roi_data['point_mm'] = [i.text for i in all_points[0].findall("string")]
            roi_data['point_px'] = [i.text for i in all_points[1].findall("string")]
            
            image_data['ROIs'].append(roi_data)
        all_images.append(image_data)

    return all_images

In [107]:
def create_segments(image_array, points):
    polygon = Polygon(points, closed=True, edgecolor='r', facecolor='r')
    polygon_indices = np.array(points)
    polygon_indices[:, 0] = np.clip(polygon_indices[:, 0], 0, 511)
    polygon_indices[:, 1] = np.clip(polygon_indices[:, 1], 0, 511)
    image_array[polygon_indices[:, 1], polygon_indices[:, 0]] = 1
    polygon_path = Path(polygon_indices)
    x, y = np.meshgrid(np.arange(512), np.arange(512))
    points = np.column_stack((x.flatten(), y.flatten()))
    mask = polygon_path.contains_points(points).reshape(512, 512)
    image_array[mask] = 1

    return image_array

In [108]:
class CTDataset(Dataset):
    def __init__(self, dir):
        super().__init__()
        self.dir = dir
        self.images_path = join(dir, "images")
        self.labels_path = join(dir, "labels")
        self.images = os.listdir(self.images_path)
        self.labels = os.listdir(self.labels_path)

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx): # Return tuple (x, y)
        try:
            img = np.load(join(self.images_path, self.images[idx]), allow_pickle=True)
            img = img.reshape(1, 512, 512) # Hard coded since all images are 512, 512

            label = np.load(join(self.labels_path, self.labels[idx]), allow_pickle=True)
            label = label.reshape(1, 512, 512)
        except:
            img, label = np.zeros((1,512,512), np.float32),np.zeros((1,512,512), np.float32)
        return  img,label

In [109]:
train = CTDataset("Numpy Dataset\\train")
valid = CTDataset("Numpy Dataset\\valid")
test = CTDataset("Numpy Dataset\\test")

In [110]:
trainDL = DataLoader(train, batch_size=CFG.batch_size,shuffle=True, pin_memory=True)
validDL = DataLoader(valid, batch_size=CFG.batch_size)
testDL = DataLoader(test, batch_size=1)

In [111]:
print(f"Train: {len(train)}")
print(f"Valid: {len(valid)}")
print(f"Test: {len(test)}")

Train: 2380
Valid: 1245
Test: 377


# Model

In [112]:
class Block(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()

        self.conv1 = Conv2d(in_ch, out_ch, 3, padding=1)
        self.relu  = ReLU()
        self.conv2 = Conv2d(out_ch, out_ch, 3, padding=1)

        # init random weights
        nn.init.xavier_normal_(self.conv1.weight)
        nn.init.zeros_(self.conv1.bias)

        nn.init.xavier_normal_(self.conv2.weight)
        nn.init.zeros_(self.conv2.bias)
    
    def forward(self, x):
        return self.relu(self.conv2(self.relu(self.conv1(x))))

In [113]:
class AutoEncoder(nn.Module):
    def __init__(self, channels = (1,32,64,128,256,512,1024)):
        super().__init__()
        self.channels = channels
        self.pool = MaxPool2d((2,2))

        self.encoder = nn.ModuleList([
            Block(channels[i], channels[i+1]) for i in range(len(channels)-1) # 1, 32, ..., 1024
        ])

    def forward(self, x):
        skip_out = []
        for block in self.encoder: # Goes through all blocks, passes through block and saves skip output
            x = block(x)
            skip_out.append(x)
            x = self.pool(x) # Reduces dim
        return skip_out

In [114]:
class AutoDecoder(nn.Module):
    def __init__(self, channels = (1,32,64,128,256,512,1024)):
        super().__init__()
        self.channels = channels[:0:-1] # Reverse of Encoder (Excluding First Unneeded in Output)
        self.pool = MaxPool2d((2,2))
        self.upconv = nn.ModuleList([
            ConvTranspose2d(self.channels[i], self.channels[i+1], 2, 2) for i in range(len(self.channels)-1)
        ])

        self.decoder = nn.ModuleList([
            Block(self.channels[i], self.channels[i+1]) for i in range(len(self.channels)-1)
        ])

    def center_crop(self, x, enc_out): # Crop encoder output
        _, _, h, w = x.shape
        enc_out = CenterCrop([h,w])(enc_out)
        return enc_out
    
    def forward(self, x, enc_out:list):
        for i in range(len(self.channels)-1):
            x = self.upconv[i](x)
            enc_ftrs = self.center_crop(x, enc_out[i]) # Crop Skip
            x = torch.cat([x, enc_ftrs], dim=1) # Concatenate Decoder and Skip
            x = self.decoder[i](x)

            # Min Max Scaling [0,1]
            x = (x-x.min())/(x.max()-x.min())
        return x


In [115]:
class UNET(nn.Module):
    def __init__(self, channels = (1,32,64,128,256,512,1024)):
        super().__init__()

        # Encoder Path
        self.enc_path = AutoEncoder(channels)

        # Decoder Path
        self.dec_path = AutoDecoder(channels)

        self.out = Conv2d(channels[1], 1, 1)

        # init random weights
        nn.init.xavier_normal_(self.out.weight)
        nn.init.zeros_(self.out.bias)

    
    def forward(self, x):
        skips = self.enc_path(x)
        x = self.dec_path(skips[::-1][0], skips[::-1][1:]) 
        # Reverse of enc_out = upward path of decoder 
        #  [0] -> 1024 output
        # [1:] -> All other skip outputs
        x = self.out(x)
        # x = F.interpolate(x, (512,512))
        x = F.sigmoid(x)

        return x

In [116]:
model = UNET().to(CFG.device)

In [117]:
if not CFG.train_new:
    try:
        model = torch.load(f"Models/{CFG.model_name}/{CFG.model_name}.pt")
    except FileNotFoundError:
        print("Model not found in models folder")

# Train

In [118]:
print(CFG.model_name)

baseline_10


In [119]:
optim = AdamW(model.parameters(), lr=CFG.lr) # AdamW

## Dice Loss

Sørensen–Dice coefficient:
[Wikipedia](https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient)
</br>
$${\displaystyle DSC={\frac {2|X\cap Y|}{|X|+|Y|}}}

In [120]:
class diceCoef(nn.Module):
    def init(self):
        super(diceCoef, self).init()

    def forward(self, inputs, targets, smooth=1):        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        return 1 - dice

In [121]:
class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss
        
        return Dice_BCE

In [122]:
class IoULoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(IoULoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        #intersection is equivalent to True Positive count
        #union is the mutually inclusive area of all labels & predictions 
        intersection = (inputs * targets).sum()
        total = (inputs + targets).sum()
        union = total - intersection 
        
        IoU = (intersection + smooth)/(union + smooth)
                
        return 1 - IoU

In [123]:
criterion_dice = DiceBCELoss()

In [124]:
criterion_iou = IoULoss()

* Pixel Binary
* IOU
* BCE Dice

## Hyperparameter Tuning

hyperparameter_ranges = {
    'lr': (0.001, 0.3),
    'batch_size': (16, 32),
}

# Step 3: Define the search space
search_space = {
    'lr': lambda: 10 ** (-3 * torch.rand(1)),
    'batch_size': lambda: torch.randint(16, 33, (1,)).item(),
}

def train_model(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(CFG.device), target.to(CFG.device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion_dice(output, target)
        loss.backward()
        optimizer.step()

def evaluate_model(model, val_loader, b_size):
    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(CFG.device), target.to(CFG.device)
            output = model(data)
            val_loss += criterion_dice(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    val_loss /= len(val_loader.dataset)
    accuracy = correct / (len(val_loader.dataset)//b_size)
    return val_loss, accuracy

num_random_samples = 5
best_accuracy = 0
best_loss = 1000
best_hyperparameters = None
weights = None
for _ in range(num_random_samples):
    hyperparameters = {param: sampler() for param, sampler in search_space.items()}
    model = UNET().to(CFG.device)
    optimizer = AdamW(model.parameters(), lr=float(hyperparameters['lr']))
    train_loader = DataLoader(train, batch_size=hyperparameters['batch_size'], shuffle=True)
    val_loader = DataLoader(valid, batch_size=hyperparameters['batch_size'], shuffle=False)

    for epoch in range(5): # Run 5 epochs on each
        train_model(model, train_loader, optimizer)
    
    val_loss, accuracy = evaluate_model(model, val_loader,hyperparameters['batch_size'])
    if val_loss < best_loss:
        weights = model.state_dict
        best_accuracy = accuracy
        best_hyperparameters = hyperparameters
        best_loss = val_loss

print("Best hyperparameters:", best_hyperparameters)
print("Best accuracy:", best_accuracy)
print("Lowest Loss:", best_loss)

## Training Loop

In [125]:
history = {"train":[],
           "valid":[]}

model = UNET().to(CFG.device)
optim = AdamW(model.parameters(), lr=float(best_hyperparameters['lr']))
trainDL = DataLoader(train, batch_size=best_hyperparameters['batch_size'], shuffle=True)
validDL = DataLoader(valid, batch_size=best_hyperparameters['batch_size'], shuffle=False)

In [126]:
model.train()
for e in range(CFG.nEpochs):
    print(f"[INFO] Epoch {e+1}/{CFG.nEpochs}")
    
    train_loss = 0
    dice_loss = 0
    iou_loss = 0
    for x,y in tqdm(trainDL):
        x, y = x.to(CFG.device), y.to(CFG.device)
        optim.zero_grad()
        pred = model(x)
        loss = criterion_dice(pred, y)
        loss.backward()
        optim.step()
        train_loss += loss
    # eval
    with torch.no_grad():
        for x,y in validDL:
            x, y = x.to(CFG.device), y.to(CFG.device)
            pred = model(x)
            loss_dice = criterion_dice(pred, y)
            loss_iou = criterion_iou(pred, y)
            dice_loss += loss_dice
            iou_loss += loss_iou

        
    avg_train_loss = train_loss/len(trainDL)
    avg_dice_loss = dice_loss/len(validDL)
    avg_iou_loss = iou_loss/len(validDL)
    
    print(f"Train Loss: {avg_train_loss}")
    print(f"Validation Dice Loss: {avg_dice_loss}")
    print(f"Validation IOU Loss: {avg_iou_loss}")
    history["train"].append(avg_train_loss)
    history['valid'].append(avg_dice_loss)

[INFO] Epoch 1/10


100%|██████████| 298/298 [03:43<00:00,  1.33it/s]


Train Loss: 1.701959252357483
Validation Dice Loss: 1.6927014589309692
Validation IOU Loss: 0.9996717572212219
[INFO] Epoch 2/10


100%|██████████| 298/298 [03:26<00:00,  1.45it/s]


Train Loss: 1.6927334070205688
Validation Dice Loss: 1.6924914121627808
Validation IOU Loss: 0.9996717572212219
[INFO] Epoch 3/10


 48%|████▊     | 143/298 [02:00<02:10,  1.19it/s]


KeyboardInterrupt: 

# Save Model

Create folder 


Models\\model_name\\

For Each Training
Models\\baseline_0\\baseline_0.pt

In [ ]:
if CFG.train_new:
    os.mkdir(f"Models/{CFG.model_name}")
    os.mkdir(f"Models/{CFG.model_name}/logs")

In [ ]:
# Save History as JSON
log_num = len(os.listdir(f"Models/{CFG.model_name}/logs"))
with open(f"Models/{CFG.model_name}/logs/log_{log_num}", "w") as out_path:
    json.dump(history, out_path)

TypeError: Object of type Tensor is not JSON serializable

In [ ]:
torch.save(model, f"Models/{CFG.model_name}/{CFG.model_name}.pt")

In [ ]:
CFG.model_name

'baseline_1'

# Notes

* Dropout
* Pixel Level Accuracy
* Single Conv in Block (Save Memory)
* Check Zoom
* 3D UNET